In [ ]:
# Etapa 1 – Importações

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Etapa 2 – Carregar Dataset com TensorFlow Datasets

In [ ]:
import tensorflow_datasets as tfds
(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

In [ ]:
# Etapa 3 – Pré-processamento (padronização e redimensionamento)

In [ ]:
def format_image(image, label):
    image = tf.image.resize(image, (160, 160))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_train.map(format_image).batch(32).prefetch(1)
ds_val = ds_val.map(format_image).batch(32).prefetch(1)

In [ ]:
# Etapa 4 – Carregar Modelo Pré-Treinado

In [ ]:
base_model = MobileNetV2(input_shape=(160, 160, 3), include_top=False, weights='imagenet')
base_model.trainable = False
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Etapa 5 – Compilar e Treinar o Modelo

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(ds_train, validation_data=ds_val, epochs=5, callbacks=[early_stop])

In [ ]:
# Etapa 6 – Avaliação e Visualização

In [ ]:
plt.plot(history.history['accuracy'], label='Acurácia Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia Validação')
plt.legend()
plt.title('Acurácia')
plt.show()

In [ ]:
# Etapa 7 – Salvar Modelo

In [ ]:
model.save('transfer_model.h5')